In [172]:
import geodb.model
from geodb.model import GPSPoint, db_url, GPSTrack, clone_model

In [173]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [174]:
engine = sqlalchemy.create_engine(db_url(), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [175]:
import pandas as pd
import numpy as np

In [176]:
query = """
select date(time), count(*) from point
where time > '2019-11-01'
group by 1
order by 1
"""
display(pd.read_sql_query(query, engine))

,date,count
0,2019-11-01,1
1,2019-11-02,2
2,2019-11-03,1
3,2019-11-04,1
4,2019-11-05,1
5,2019-11-06,2
6,2019-11-07,1
7,2019-11-09,3
8,2019-11-10,3
9,2019-11-11,1


In [177]:
date = "2019-11-23"

In [178]:
redundant_waypoint_query = """
select name, time, count(*), max(track.id) as id from track
join point on track.id = track_id
where type = 'Waypoint'
group by 1, 2
having count(*) > 1
order by 3 desc
"""
display(pd.read_sql_query(redundant_waypoint_query, engine))

,name,time,count,id


In [179]:
redundant = [session.query(GPSTrack).get(int(p)) for p in pd.read_sql_query(redundant_waypoint_query, engine)[['id']].values]
len(redundant)

0

In [180]:
for r in redundant:
    if r is not None:
        session.delete(r)
session.commit()

In [181]:
bracket_query = """
select track.id, min(time) as min_time, max(time) as max_time from track
join point on track_id = track.id
group by track.id
having abs(date(min(time)) - %(date)s) < 2  or abs(date(max(time)) - %(date)s) < 2
"""
display(pd.read_sql_query(bracket_query, engine, params={'date': date}))


,id,min_time,max_time
0,9890,2019-11-23 16:16:55+00:00,2019-11-23 16:17:06+00:00
1,44,2019-11-23 13:35:44+00:00,2019-11-23 13:35:44+00:00
2,2906,2019-11-23 10:10:28+00:00,2019-11-23 10:10:28+00:00
3,99,2019-11-23 16:15:08+00:00,2019-11-23 16:15:08+00:00
4,9924,2019-11-24 11:14:41+00:00,2019-11-24 11:14:41+00:00
...,...,...,...
171,104,2019-11-23 16:16:25+00:00,2019-11-23 16:16:31+00:00
172,2487,2019-11-24 23:33:45+00:00,2019-11-25 09:07:48+00:00
173,9910,2019-11-24 08:21:40+00:00,2019-11-24 08:24:52+00:00
174,9898,2019-11-23 18:09:23+00:00,2019-11-23 18:09:23+00:00


In [182]:
bracket_tracks = [session.query(GPSTrack).get(int(t)) for t in pd.read_sql_query(bracket_query, engine, params={'date': date})[['id']].values]
len(bracket_tracks)

176

In [183]:
day_tracks = [t for t in bracket_tracks if str(t.start.localtime.date()) == date or str(t.end.localtime.date()) == date]
len(day_tracks)

73

In [184]:
b = list(day_tracks[0].bounds)
for t in day_tracks:
    tb = t.bounds
    b[0] = min(b[0], tb[0])
    b[1] = min(b[1], tb[1])
    b[2] = max(b[2], tb[2])
    b[3] = max(b[3], tb[2])
    
center = (b[1] + b[3]/2, (b[0] + b[2])/2)

In [185]:
set([t.name for t in day_tracks])

{'AAL2518-20191123',
 'AAL61-20191123',
 'Austin Bergstrom International Airport (AUS)',
 'Gate A23',
 None,
 'Pappasito&apos;s Cantina',
 "Pappasito's Cantina",
 'Skytram'}

In [186]:
from intervaltree import IntervalTree
from datetime import timedelta

In [187]:
intervals = IntervalTree()
for t in day_tracks:
    start = t.start.time
    end = t.end.time
    if start == end:
        end += timedelta(seconds=1)
    intervals.addi(start, end, [t])
intervals.split_overlaps()
intervals.merge_equals(data_reducer=lambda a, b: a+b)

In [188]:
for i in sorted(intervals):
    print(f"{str(i.begin)}\t{str(i.end)}\t{i.end - i.begin}\t{[t.name or t.filename for t in i.data]}")

2019-11-23 09:36:56+00:00	2019-11-23 09:56:01+00:00	0:19:05	['2019-11-23-edited.gpx']
2019-11-23 09:56:12+00:00	2019-11-23 09:56:13+00:00	0:00:01	['Austin Bergstrom International Airport (AUS)']
2019-11-23 10:10:28+00:00	2019-11-23 10:10:29+00:00	0:00:01	['Austin Bergstrom International Airport (AUS)']
2019-11-23 10:14:46+00:00	2019-11-23 10:19:44+00:00	0:04:58	['2019-11-23-edited.gpx']
2019-11-23 10:20:06+00:00	2019-11-23 10:20:07+00:00	0:00:01	['Austin Bergstrom International Airport (AUS)']
2019-11-23 11:53:20+00:00	2019-11-23 12:29:11+00:00	0:35:51	['AAL2518-20191123', 'AAL2518-20191123']
2019-11-23 12:47:51+00:00	2019-11-23 12:47:52+00:00	0:00:01	['Skytram']
2019-11-23 12:50:46+00:00	2019-11-23 13:03:40+00:00	0:12:54	['2019-11-23-edited.gpx']
2019-11-23 13:03:44+00:00	2019-11-23 13:03:45+00:00	0:00:01	["Pappasito's Cantina", 'Pappasito&apos;s Cantina']
2019-11-23 13:07:46+00:00	2019-11-23 13:07:47+00:00	0:00:01	["Pappasito's Cantina"]
2019-11-23 13:30:55+00:00	2019-11-23 13:35:29+

In [189]:
import gpxpy

In [193]:
gpx = gpxpy.gpx.GPX()
for i in sorted(intervals):
    track_ids = [t.id for t in i.data]
    points = (
        session.query(GPSPoint)
        .filter(GPSPoint.track_id.in_(track_ids))
        .filter(GPSPoint.time.between(i.begin, i.end))
        .order_by(GPSPoint.time)
    ).all()
    
    if len(points) == 1:
        p = points[0]
        t = p.track
        w = gpxpy.gpx.GPXWaypoint(
            latitude=p.latitude, longitude=p.longitude, elevation=p.elevation,
            time=p.time, name=t.name, description=t.description, type=t.type,
            comment=t.source or t.filename
        )
        gpx.waypoints.append(w)
    else:
        t = gpxpy.gpx.GPXTrack(
            name=f"{points[0].localtime.time()} {' '.join({t.type for t in i.data})}",
            description="\t".join({t.description or "" for t in i.data}),
        )
        t.comment="\t".join({t.name or t.filename or "" for t in i.data})
        t.source="\t".join({t.source or "" for t in i.data})
        t.type="\t".join({t.type or "" for t in i.data})
        s = gpxpy.gpx.GPXTrackSegment()
        t.segments.append(s)
        for p in points:
            s.points.append(gpxpy.gpx.GPXTrackPoint(
                latitude=p.latitude, longitude=p.longitude, elevation=p.elevation, time=p.time))
        gpx.tracks.append(t)
    print(f"{str(points[0].localtime)}\t{str(points[-1].localtime.time())}\t{i.end - i.begin}\t{len(points)}\t{{t.name or t.filename for t in i.data}")
    
    

2019-11-23 03:36:56-06:00	03:56:01	0:19:05	82	['2019-11-23-edited.gpx']
2019-11-23 03:56:12-06:00	03:56:12	0:00:01	1	['Austin Bergstrom International Airport (AUS)']
2019-11-23 04:10:28-06:00	04:10:28	0:00:01	1	['Austin Bergstrom International Airport (AUS)']
2019-11-23 04:14:46-06:00	04:19:44	0:04:58	42	['2019-11-23-edited.gpx']
2019-11-23 04:20:06-06:00	04:20:06	0:00:01	1	['Austin Bergstrom International Airport (AUS)']
2019-11-23 05:53:20-06:00	06:29:11	0:35:51	182	['AAL2518-20191123', 'AAL2518-20191123']
2019-11-23 06:47:51-06:00	06:47:51	0:00:01	1	['Skytram']
2019-11-23 06:50:46-06:00	07:03:40	0:12:54	97	['2019-11-23-edited.gpx']
2019-11-23 07:03:44-06:00	07:03:44	0:00:01	2	["Pappasito's Cantina", 'Pappasito&apos;s Cantina']
2019-11-23 07:07:46-06:00	07:07:46	0:00:01	1	["Pappasito's Cantina"]
2019-11-23 07:30:55-06:00	07:35:29	0:04:34	57	['2019-11-23-edited.gpx']
2019-11-23 07:35:44-06:00	07:35:44	0:00:01	1	['Gate A23']
2019-11-23 07:38:58-06:00	07:40:21	0:01:23	23	['2019-11-23-ed

In [194]:
with open(f"{date}.gpx", "w") as fh:
    print(gpx.to_xml(version="1.1"), file=fh)

In [195]:
query = """
select * from track
where name = 'AAL61-20191123'
"""
display(pd.read_sql_query(query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,2515,AAL61-20191123,None,KDFW - RJAA,None,airplane,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,{},None
1,12373,AAL61-20191123,None,KDFW - RJAA,None,airplane,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,{},None


In [168]:
query = """
select track_id, count(*) from point
where tz is null
group by track_id
"""
display(pd.read_sql_query(query, engine))

,track_id,count
0,None,6349


In [171]:
query = """
select count(*) from point
where track_id is null
"""
display(pd.read_sql_query(query, engine))

,count
0,0


In [170]:
session.execute("""delete from point where track_id is null""")
session.commit()

In [35]:
merged = GPSTrack(name=f"Merged {date}")
points = set()
for t in day_tracks:
    for p in t.points:
        if p.time not in points:
            points.add(p.time)
            clone = clone_model(p)
            clone.properties["original_track"] = t.id
            clone.properties["original_type"] = t.type
            merged.points.append(clone)
session.add(merged)
session.commit()

In [36]:
merged = session.query(GPSTrack).filter(GPSTrack.name==f"Merged {date}").one()
bounds = merged.bounds
center = ((bounds[1] + bounds[3])/2, (bounds[0] + bounds[2])/2)

In [37]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,
    Polyline, 
    Popup,
    GeoJSON,
    DrawControl
)
from ipywidgets import HTML

from traitlets import link

In [38]:
def marker(t):
    marker = Marker(location=t.points[0].lat_lon)
    marker.popup = HTML(f"{t.id}: {t.name}")
    return marker

def add_track(m, t, **kwargs):
        l = t.as_polyline(
            fill=False,
            **kwargs
        )
        m.add_layer(l)
        l.popup = HTML(f"{t.id}: {t.name or t.filename}<br/>{t.points[0].time} - {t.points[-1].time}")

In [39]:
m = Map(center=center, zoom=2, close_popup_on_click=False)

markers = []
for track in day_tracks:
    if len(track.points) == 1 and track.name is not None:
        markers.append(marker(track))
    else:
        add_track(m, track)
add_track(m, merged, color='red')
m.add_layer(MarkerCluster(markers=markers))

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [40]:
merged.id

12433

In [41]:
discontinuity_query = """
select * from (
select
time,
properties->'original_track' as orig, 
lead(properties->'original_track', 1) over (order by time) as next,
latitude, longitude,
lead(latitude, 1) over (order by time) as lat,
lead(longitude, 1) over (order by time) as long,
rank() over (order by time)
from point
where track_id = 12431
and time > '2019-11-23 11:30:20'
and time < '2019-11-23 12:50:11'
) x where orig != next
"""
display(pd.read_sql_query(discontinuity_query, engine))

,time,orig,next,latitude,longitude,lat,long,rank


In [155]:
session.execute("""delete from track where name like '%Merged%'""")
session.commit()

In [43]:
# This is probably not the way we want to calculate splits
splits_query = """
select * from (
select *,
lead(orig, 1) over (order by time) as next,
rank() over (order by time)
from (
select point.id, time, track.id as orig, name, filename, type
from point
join track on track.id = (point.properties->'original_track')::integer
where track_id = %(merged)s
-- and track.type != 'Waypoint'
) x
) y
where orig != next
order by rank
"""
display(pd.read_sql_query(splits_query, engine, params={'merged': merged.id}))

,id,time,orig,name,filename,type,next,rank
0,234574,2019-11-23 00:37:58+00:00,12406,None,2019-11-23-edited.gpx,car,12407,103
1,234133,2019-11-23 01:21:50+00:00,12407,None,2019-11-23-edited.gpx,walking,12408,163
2,234073,2019-11-23 01:31:21+00:00,12408,None,2019-11-23-edited.gpx,stationary,12409,242
3,234413,2019-11-23 09:56:01+00:00,12409,None,2019-11-23-edited.gpx,car,40,321
4,234304,2019-11-23 09:56:12+00:00,40,Austin Bergstrom International Airport (AUS),None,Waypoint,2906,322
...,...,...,...,...,...,...,...,...
703,231482,2019-11-24 04:30:35+00:00,2515,AAL61-20191123,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,airplane,9899,2882
704,233360,2019-11-24 05:15:21+00:00,9899,None,2019-11-24.gpx,airplane,2515,2952
705,231483,2019-11-24 05:15:24+00:00,2515,AAL61-20191123,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,airplane,9899,2953
706,233529,2019-11-24 05:59:20+00:00,9899,None,2019-11-24.gpx,airplane,2515,3122


In [44]:
discontinuity_query = """
select * from track where filename like '%%Flight%%'
"""
display(pd.read_sql_query(discontinuity_query, engine))

,id,name,comment,description,source,type,filename,properties,raw
0,2504,AAL1584-20191210,None,KAUS - KDFW,None,airplane,FlightAware_AAL1584_KAUS_KDFW_20191210.gpx,{},None
1,2508,AAL176-20191210,None,RJAA - KDFW,None,airplane,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None
2,2509,20191210,None,None,None,airplane,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None
3,2512,AAL2518-20191123,None,KAUS - KDFW,None,airplane,FlightAware_AAL2518_KAUS_KDFW_20191123.gpx,{},None
4,2515,AAL61-20191123,None,KDFW - RJAA,None,airplane,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,{},None
5,2516,20191124,None,None,None,airplane,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,{},None
6,12366,AAL176-20191210,None,RJAA - KDFW,None,airplane,FlightAware_AAL176_RJAA_KDFW_20191210.gpx,{},None
7,12373,AAL61-20191123,None,KDFW - RJAA,None,airplane,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,{},None
8,12374,20191124,None,None,None,airplane,FlightAware_AAL61_KDFW_RJAA_20191123.gpx,{},None
9,12363,AAL1584-20191210,None,KAUS - KDFW,None,airplane,FlightAware_AAL1584_KAUS_KDFW_20191210.gpx,{},None


In [45]:
flight_query = """update track set type = 'airplane' where filename like '%%Flight%%'"""
session.rollback()
session.execute(flight_query)
session.commit()

In [46]:
set([t.filename for t in day_tracks])

{'2019-11-23-edited.gpx',
 '2019-11-24.gpx',
 'FlightAware_AAL2518_KAUS_KDFW_20191123.gpx',
 'FlightAware_AAL61_KDFW_RJAA_20191123.gpx',
 None}

In [59]:
splits = sorted(set([p for l in [[t.points[0].time, t.points[-1].time] for t in day_tracks] for p in l]))
split_points = []
current_split = []
for p in merged.points:
    if not splits or p.time < splits[0]:
        current_split.append(p)
        continue
    current_split.append(p)
    split_points.append(current_split)
    current_split = []
    splits.pop(0)
else:
    if current_split:
        split_points.append(current_split)
        
len(split_points)
    
    

74

In [60]:
len(merged.points)

3584

In [64]:
[len(s) for s in split_points]

[1,
 102,
 1,
 59,
 1,
 78,
 1,
 78,
 1,
 1,
 1,
 41,
 1,
 1,
 90,
 1,
 1,
 94,
 1,
 1,
 1,
 56,
 1,
 1,
 22,
 1,
 1,
 14,
 1,
 1,
 1,
 1,
 13,
 1,
 1,
 1,
 1,
 15,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 4,
 1,
 1,
 3,
 1,
 1,
 17,
 1,
 1,
 42,
 1,
 7,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 71,
 2257,
 461]

In [62]:
[set([p.properties["original_type"] for p in s if p.properties["original_type"]]) for s in split_points]

[{'car'},
 {'car'},
 {'walking'},
 {'walking'},
 {'stationary'},
 {'stationary'},
 {'car'},
 {'car'},
 {'Waypoint'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'Waypoint'},
 {'airplane'},
 {'airplane'},
 {'Waypoint'},
 {'tram'},
 {'tram'},
 {'Waypoint'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'Waypoint'},
 {'airplane'},
 {'stationary'},
 {'walking'},
 {'walking'},
 {'airplane'},
 {'stationary'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'stationary'},
 {'walking'},
 {'walking'},
 {'airplane'},
 {'airplane'},
 {'stationary'},
 {'walking'},
 {'walking'},
 {'stationary'},
 {'stationary'},
 {'walking'},
 {'walking'},
 {'stationary'},
 {'walking'},
 {'walking'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'Waypoint'},
 {'walking'},
 {'walking'},
 {'airplane'},
 {'airplane'},
 {'stationary'},
 {'stationary'},
 {'airplane'},
 {'airplane'},
 {'Waypoint'},
 {'stationary'},
 {'airplane'},
 {'airplan

In [63]:
set([p.properties["original_type"] for p in merged.points])

{'Waypoint', 'airplane', 'car', 'stationary', 'tram', 'walking'}